# Pydeck + Earth Engine: Terrain Visualization

This is an example of using [pydeck](https://pydeck.gl) to visualize a Google Earth Engine `Image` object _over 3D terrain_. To install and run this notebook locally, refer to the [Pydeck Earth Engine documentation](https://earthengine-layers.com/docs/developer-guide/pydeck-integration).

To see this example online, view the [JavaScript version][js-example].

[js-example]: https://earthengine-layers.com/examples/terrain

First, import required packages. Note that here we import the `EarthEngineTerrainLayer` instead of the `EarthEngineLayer`.

In [ ]:
from pydeck_earthengine_layers import EarthEngineTerrainLayer
import pydeck as pdk
import ee

### Authenticate with Earth Engine

Using Earth Engine requires authentication. If you don't have a Google account approved for use with Earth Engine, you'll need to request access. For more information and to sign up, go to https://signup.earthengine.google.com/.

In [ ]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

### Terrain Example

In contrast to the `EarthEngineLayer`, where you need to supply _one_ Earth Engine object to render, with the `EarthEngineTerrainLayer` you must supply **two** Earth Engine objects. The first is used to render the image in the same way as the `EarthEngineLayer`; the second supplies elevation values used to extrude terrain in 3D. Hence the former can be any `Image` object; the latter must be an `Image` object whose values represents terrain heights.

It's important for the terrain source to have relatively high spatial resolution. In previous examples, we used [SRTM90][srtm90] as an elevation source, but that only has a resolution of 90 meters. When used as an elevation source, it looks very blocky/pixelated at high zoom levels. In this example we'll use [SRTM30][srtm30] (30-meter resolution) as the `Image` source and the [USGS's National Elevation Dataset][ned] (10-meter resolution, U.S. only) as the terrain source. SRTM30 is generally the best-resolution worldwide data source available.

[srtm90]: https://developers.google.com/earth-engine/datasets/catalog/CGIAR_SRTM90_V4
[srtm30]: https://developers.google.com/earth-engine/datasets/catalog/USGS_SRTMGL1_003
[ned]: https://developers.google.com/earth-engine/datasets/catalog/USGS_NED

In [ ]:
image = ee.Image('USGS/SRTMGL1_003')
terrain = ee.Image('USGS/NED').select('elevation')

Here `vis_params` consists of parameters that will be passed to the Earth Engine [`visParams` argument][visparams]. Any parameters that you could pass directly to Earth Engine in the code editor, you can also pass here to the `EarthEngineLayer`.

[visparams]: https://developers.google.com/earth-engine/image_visualization

In [ ]:
vis_params={
    "min": 0, 
    "max": 4000,
    "palette": ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
}

Now we're ready to create the Pydeck layer. The `EarthEngineLayer` makes this simple. Just pass the Earth Engine object to the class.

Including the `id` argument isn't necessary when you only have one pydeck layer, but it is necessary to distinguish multiple layers, so it's good to get into the habit of including an `id` parameter.

In [ ]:
ee_layer = EarthEngineTerrainLayer(
    image,
    terrain,
    vis_params,
    id="EETerrainLayer"
)

Then just pass this layer to a `pydeck.Deck` instance, and call `.show()` to create a map:

In [ ]:
view_state = pdk.ViewState(
    latitude=36.15,
    longitude=-111.96,
    zoom=10.5, 
    bearing=-66.16,
    pitch=60)
r = pdk.Deck(
    layers=[ee_layer], 
    initial_view_state=view_state
)
r.show()